In [1]:
# Se necessário, descomente a próxima linha:
# !pip install pandas numpy plotly scikit-learn textwrap3

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from textwrap import dedent
from pathlib import Path

PD_OPTS = dict(float_format=lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X","."))
OUTPUT_DIR = Path("./")


In [2]:
# Ajuste o caminho se o arquivo estiver no Drive
CSV_PATH = "dataset_ecommerce_2024.csv"

# Tentativa padrão
try:
    df = pd.read_csv(CSV_PATH)
except UnicodeDecodeError:
    # Caso haja mojibake (ex.: NiterÃ³i), tenta latin-1 e converte
    df = pd.read_csv(CSV_PATH, encoding="latin1")

# Padroniza nomes de colunas
df.columns = [c.strip().lower() for c in df.columns]

# Corrige possíveis caracteres quebrados comuns (opcional)
for col in ['city','state','channel','product_category','carrier','payment_method']:
    if col in df.columns and df[col].dtype == object:
        try:
            df[col] = df[col].apply(lambda s: s.encode('latin1').decode('utf-8') if isinstance(s,str) else s)
        except Exception:
            pass

print(df.shape)
df.head()


(600, 20)


,order_id,order_date,customer_id,city,state,channel,product_category,unit_price,quantity,discount_pct,shipping_cost,carrier,payment_method,declared_weight,actual_weight,days_in_transit,delivered_on,returned,rating,total_value
0,ORD100000,2024-04-12,C7731,Niterói,RJ,Mobile App,Sports,956.08,1,0.111,51.79,Correios,Boleto,1.251,1.311,4,2024-04-16,0,5,1007.87
1,ORD100001,2024-12-14,C2810,São Paulo,SP,Marketplace,Fashion,278.91,1,0.103,44.76,Total Express,Debit Card,0.295,0.287,3,2024-12-17,0,3,323.67
2,ORD100002,2024-09-27,C1133,Curitiba,PR,Marketplace,Beauty,337.18,1,0.047,17.67,Jadlog,Cash,0.337,0.365,2,2024-09-29,0,5,354.85
3,ORD100003,2024-04-16,C6063,Guarulhos,SP,Web,Electronics,927.93,1,0.045,16.79,Loggi,Credit Card,0.883,0.824,4,2024-04-20,0,5,944.72
4,ORD100004,2024-03-12,C8026,Porto Alegre,RS,Marketplace,Fashion,50.16,4,0.101,22.85,Jadlog,Cash,0.063,0.064,1,2024-03-13,0,3,223.49


In [3]:
df_raw = df.copy()

# 2.1 Datas
for c in ["order_date","delivered_on"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce')

# 2.2 Numéricos garantidos
num_cols = ["unit_price","quantity","discount_pct","shipping_cost","declared_weight","actual_weight",
            "days_in_transit","rating","total_value"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# 2.3 Remove linhas sem info essencial
must_have = ["order_date","unit_price","quantity","total_value"]
df = df.dropna(subset=[c for c in must_have if c in df.columns]).copy()

# 2.4 Filtros de validade
if "unit_price" in df: df = df[df["unit_price"] >= 0]
if "quantity" in df:   df = df[df["quantity"]  >  0]

# 2.5 Derivadas
if "order_date" in df:
    df["month"] = df["order_date"].dt.to_period("M").astype(str)

# ticket médio por item vendido na linha
df["ticket"] = df["total_value"]/df["quantity"]

# 2.6 Outliers por IQR
def mark_outliers_iqr(s: pd.Series) -> pd.Series:
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    low, up = q1 - 1.5*iqr, q3 + 1.5*iqr
    return (s < low) | (s > up)

for col in ["unit_price","total_value","ticket"]:
    if col in df.columns:
        df[f"out_{col}"] = mark_outliers_iqr(df[col])

# 2.7 Exporta dataset limpo
clean_path = OUTPUT_DIR / "clean_dataset.csv"
df.to_csv(clean_path, index=False)
clean_path, df.shape


(PosixPath('clean_dataset.csv'), (600, 25))

In [4]:
results = {}

# 3.1 Receita por canal
if "channel" in df and "total_value" in df:
    rev_channel = (df.groupby("channel", as_index=False)["total_value"]
                     .sum()
                     .sort_values("total_value", ascending=False))
    results["top_channel"] = rev_channel.iloc[0].to_dict() if len(rev_channel) else None
else:
    rev_channel = pd.DataFrame()

# 3.2 Pareto por categoria
if "product_category" in df and "total_value" in df:
    cat_rev = (df.groupby("product_category", as_index=False)["total_value"]
                 .sum()
                 .sort_values("total_value", ascending=False))
    cat_rev["perc"] = cat_rev["total_value"] / cat_rev["total_value"].sum()
    cat_rev["cum_perc"] = cat_rev["perc"].cumsum()
    pareto_80 = cat_rev[cat_rev["cum_perc"]<=0.80]["product_category"].tolist()
    results["pareto_80"] = pareto_80
else:
    cat_rev = pd.DataFrame()
    results["pareto_80"] = []

# 3.3 Faturamento mensal
if "month" in df and "total_value" in df:
    rev_month = (df.groupby("month", as_index=False)["total_value"]
                   .sum()
                   .sort_values("month"))
else:
    rev_month = pd.DataFrame()

# 3.4 Velocidade (média days_in_transit) por estado
if "state" in df and "days_in_transit" in df:
    state_speed = (df.dropna(subset=["days_in_transit"])
                     .groupby("state", as_index=False)["days_in_transit"]
                     .mean()
                     .sort_values("days_in_transit"))
    results["top5_fast_states"] = state_speed.head(5)
else:
    state_speed = pd.DataFrame()
    results["top5_fast_states"] = pd.DataFrame()

# 3.5 Regressão (peso declarado vs real)
reg_summary = {}
if {"declared_weight","actual_weight"}.issubset(df.columns):
    sub = df[["declared_weight","actual_weight"]].dropna().copy()
    if len(sub) >= 2:
        X = sub[["declared_weight"]].values
        y = sub[["actual_weight"]].values
        model = LinearRegression().fit(X,y)
        pred  = model.predict(X)
        r2    = r2_score(y, pred)
        coef  = float(model.coef_.ravel()[0])
        intercept = float(model.intercept_.ravel()[0])
        reg_summary = {"coef":coef, "intercept":intercept, "r2":float(r2), "n":int(len(sub))}
    results["regression"] = reg_summary
else:
    results["regression"] = {}

# 3.6 Outliers (IQR)
out_info = {}
for col in ["unit_price","total_value","ticket"]:
    cflag = f"out_{col}"
    if cflag in df.columns:
        out_info[col] = int(df[cflag].sum())
results["outliers_iqr"] = out_info

# 3.7 Taxa de devolução (geral e por categoria)
ret_cols_ok = {"returned"}
if "returned" in df.columns:
    rate_overall = df["returned"].mean() if df["returned"].notna().any() else np.nan
    results["return_rate_overall"] = float(rate_overall) if pd.notna(rate_overall) else None
    if "product_category" in df.columns:
        ret_by_cat = (df.groupby("product_category", as_index=False)["returned"]
                        .mean()
                        .rename(columns={"returned":"return_rate"})
                        .sort_values("return_rate", ascending=False))
    else:
        ret_by_cat = pd.DataFrame()
else:
    rate_overall = np.nan
    ret_by_cat = pd.DataFrame()
    results["return_rate_overall"] = None

# 3.8 Ticket médio por canal/categoria/estado (para discussão de impacto)
def agg_ticket(col):
    if col in df.columns:
        g = df.groupby(col, as_index=False).agg(
            revenue=("total_value","sum"),
            qty=("quantity","sum"),
            ticket_mean=("ticket","mean")
        )
        g = g.sort_values("ticket_mean", ascending=False)
        return g
    return pd.DataFrame()

ticket_by_channel = agg_ticket("channel")
ticket_by_category= agg_ticket("product_category")
ticket_by_state   = agg_ticket("state")

results["ticket_by"] = {
    "channel_top": ticket_by_channel.head(3).to_dict(orient="records") if not ticket_by_channel.empty else [],
    "category_top": ticket_by_category.head(3).to_dict(orient="records") if not ticket_by_category.empty else [],
    "state_top": ticket_by_state.head(3).to_dict(orient="records") if not ticket_by_state.empty else [],
}

results


{'top_channel': {'channel': 'Web', 'total_value': 240341.89},
 'pareto_80': ['Electronics', 'Home'],
 'top5_fast_states':   state  days_in_transit
 3    RS         2.325000
 4    SC         2.515152
 1    PR         2.888298
 5    SP         4.445087
 0    MG         4.760000,
 'regression': {'coef': 1.0255604382873957,
  'intercept': -0.0020156952568094466,
  'r2': 0.9899492327579522,
  'n': 600},
 'outliers_iqr': {'unit_price': 68, 'total_value': 45, 'ticket': 66},
 'return_rate_overall': 0.05333333333333334,
 'ticket_by': {'channel_top': [{'channel': 'Physical Store',
    'revenue': 53044.229999999996,
    'qty': 87,
    'ticket_mean': 657.0585956790123},
   {'channel': 'Marketplace',
    'revenue': 98268.12,
    'qty': 175,
    'ticket_mean': 588.2819314641745},
   {'channel': 'Mobile App',
    'revenue': 150684.39,
    'qty': 276,
    'ticket_mean': 567.7043851626016}],
  'category_top': [{'product_category': 'Electronics',
    'revenue': 319745.34,
    'qty': 253,
    'ticket_mea

In [5]:
figs = []

# G1: Receita por canal
if not rev_channel.empty:
    f1 = px.bar(rev_channel, x="channel", y="total_value", title="Receita por Canal")
    figs.append(f1)

# G2: Pareto por categoria (barras + linha cumulativa)
if not cat_rev.empty:
    f2 = make_subplots(specs=[[{"secondary_y": True}]])
    f2.add_trace(go.Bar(x=cat_rev["product_category"], y=cat_rev["total_value"], name="Receita"), secondary_y=False)
    f2.add_trace(go.Scatter(x=cat_rev["product_category"], y=(cat_rev["cum_perc"]*100), name="% Acumulado", mode="lines+markers"),
                 secondary_y=True)
    f2.update_layout(title_text="Pareto de Receita por Categoria")
    f2.update_yaxes(title_text="Receita", secondary_y=False)
    f2.update_yaxes(title_text="% Acumulado", secondary_y=True, range=[0,100])
    figs.append(f2)

# G3: Faturamento mensal
if not rev_month.empty:
    f3 = px.line(rev_month, x="month", y="total_value", title="Faturamento Mensal (Sazonalidade)")
    figs.append(f3)

# G4: Top 10 estados mais rápidos (menor days_in_transit)
if not state_speed.empty:
    top10_fast = state_speed.head(10)
    f4 = px.bar(top10_fast, x="state", y="days_in_transit", title="Média de Dias em Trânsito - Top 10 mais rápidos")
    figs.append(f4)

# G5 (extra): Scatter pesos com linha de regressão
if results.get("regression") and len(results["regression"])>0:
    sub = df[["declared_weight","actual_weight"]].dropna().copy()
    f5 = px.scatter(sub, x="declared_weight", y="actual_weight", title="Peso Declarado vs Peso Real")
    # linha manual
    coef = results["regression"]["coef"]; intercept = results["regression"]["intercept"]
    xs = np.linspace(sub["declared_weight"].min(), sub["declared_weight"].max(), 100)
    ys = coef*xs + intercept
    f5.add_traces(go.Scatter(x=xs, y=ys, mode="lines", name="Regressão"))
    figs.append(f5)

# Salva um HTML com todos os gráficos concatenados
html_parts = []
for i, f in enumerate(figs, start=1):
    html_parts.append(f.to_html(full_html=False, include_plotlyjs='cdn'))
html = "<h1>Dashboard E-commerce 2024</h1>" + "\n".join(html_parts)

dash_path = OUTPUT_DIR / "dashboard.html"
dash_path.write_text(html, encoding="utf-8")
dash_path


PosixPath('dashboard.html')

In [6]:
def fmt_pct(x):
    return f"{100*x:.2f}%" if pd.notna(x) else "n/d"

# Q2: canal com maior receita
top_channel_line = "n/d"
if results.get("top_channel"):
    tc = results["top_channel"]
    top_channel_line = f'{tc["channel"]} (R$ {tc["total_value"]:,.2f})'.replace(",", "X").replace(".", ",").replace("X",".")

# Q5: top 5 estados com menor média de days_in_transit
top5_states_lines = []
if isinstance(results.get("top5_fast_states"), pd.DataFrame) and not results["top5_fast_states"].empty:
    for _,r in results["top5_fast_states"].head(5).iterrows():
        top5_states_lines.append(f'{r["state"]}: {r["days_in_transit"]:.2f} dias')

# Q6: regressão
reg = results.get("regression", {})
reg_line = "n/d"
if reg:
    reg_line = f'actual = {reg["coef"]:.4f} * declared + {reg["intercept"]:.4f}; R² = {reg["r2"]:.4f} (n={reg["n"]})'

# Q7: outliers
out_lines = [f'{k}: {v}' for k,v in results.get("outliers_iqr", {}).items()]
out_rule = "IQR (Q1-1,5*IQR ; Q3+1,5*IQR)"

# Q8: taxa de devolução
ret_overall = fmt_pct(results.get("return_rate_overall"))
ret_cat_tbl = ""
if 'ret_by_cat' not in locals():
    ret_by_cat = pd.DataFrame()
if not ret_by_cat.empty:
    ret_cat_tbl = "\n".join([f'- {row["product_category"]}: {fmt_pct(row["return_rate"])}'
                             for _,row in ret_by_cat.iterrows()])

# Q9: fatores do ticket médio (evidências simples)
def top_evidence(df_, label):
    if df_.empty: return f"- {label}: n/d"
    top = df_.head(3)[[df_.columns[0], "ticket_mean"]].values.tolist()
    items = "; ".join([f"{a}: R$ {b:.2f}" for a,b in top])
    return f"- {label} (top ticket): {items}"

ev_channel  = top_evidence(ticket_by_channel,  "Canal")
ev_category = top_evidence(ticket_by_category, "Categoria")
ev_state    = top_evidence(ticket_by_state,    "Estado")

# Q1 e Q10 – você pode editar o texto abaixo conforme o .docx de objetivos e achados
objetivos_txt = dedent("""
1) Objetivos do negócio (resumo):
- Aumentar receita por canal (priorizar canais com maior potencial).
- Reduzir prazo de entrega (otimização logística por estado/transportadora).
- Otimizar ticket médio (mix de produtos, descontos e meios de pagamento).
- Melhorar satisfação (rating) e reduzir devoluções.
""").strip()

recs_txt = dedent("""
10) Recomendações:
1. Marketing: Campanhas direcionadas no(s) canal(is) com menor receita e bom ticket médio; reforçar categorias chave do Pareto (80% do faturamento).
2. Logística: Focar nos estados com maior days_in_transit testando transportadoras alternativas e hubs regionais; SLAs por estado.
3. Atendimento/Produto: Atacar categorias com maior taxa de devolução com melhorias de descrição/fotos, QA do fornecedor e política de troca clara.
""").strip()

# Monta respostas.txt
txt = dedent(f"""
{objetivos_txt}

2) Canal com maior receita: {top_channel_line}

3) Categorias que somam ~80% do faturamento (Pareto):
- {", ".join(results.get("pareto_80", [])) if results.get("pareto_80") else "n/d"}

4) Sazonalidade (faturamento por mês): ver dashboard.html (gráfico de linha).

5) Top 5 estados com menor média de days_in_transit:
- {chr(10).join(top5_states_lines) if top5_states_lines else "n/d"}

6) Regressão (declared_weight vs actual_weight):
- {reg_line}

7) Outliers (IQR):
- Contagens -> {", ".join(out_lines) if out_lines else "n/d"}
- Regra: {out_rule}

8) Taxa de devolução:
- Geral: {ret_overall}
- Por categoria:
{ret_cat_tbl if ret_cat_tbl else "- n/d"}

9) Fatores que impactam o ticket médio (evidências):
{ev_channel}
{ev_category}
{ev_state}

{recs_txt}
""").strip()

resp_path = OUTPUT_DIR / "respostas.txt"
resp_path.write_text(txt, encoding="utf-8")
resp_path


PosixPath('respostas.txt')

In [7]:
# Tabelas que costumam ir no anexo do relatório
tables = {
    "rev_channel.csv": rev_channel,
    "pareto_category.csv": cat_rev,
    "rev_month.csv": rev_month,
    "state_days_in_transit.csv": state_speed,
    "return_rate_by_category.csv": ret_by_cat,
    "ticket_by_channel.csv": ticket_by_channel,
    "ticket_by_category.csv": ticket_by_category,
    "ticket_by_state.csv": ticket_by_state,
}
for name, df_ in tables.items():
    if isinstance(df_, pd.DataFrame) and not df_.empty:
        df_.to_csv(OUTPUT_DIR / name, index=False)
sorted(tables.keys())


['pareto_category.csv',
 'return_rate_by_category.csv',
 'rev_channel.csv',
 'rev_month.csv',
 'state_days_in_transit.csv',
 'ticket_by_category.csv',
 'ticket_by_channel.csv',
 'ticket_by_state.csv']